# TSA Chapter 3: ARIMA(p,d,q) Model Specification

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_Ch3/TSA_ch3_arima_model/TSA_ch3_arima_model.ipynb)

This notebook demonstrates:
- ARIMA model: difference d times then fit ARMA(p,q)
- ACF/PACF before and after differencing for model identification.


In [ ]:
!pip install matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

# Style configuration
COLORS = {
    'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32',
    'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none', 'figure.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False,
    'axes.spines.right': False, 'axes.grid': False, 'font.size': 9,
    'axes.titlesize': 10, 'axes.labelsize': 9, 'xtick.labelsize': 8,
    'ytick.labelsize': 8, 'legend.fontsize': 8, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.edgecolor': '#333333', 'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}.pdf + .png')

In [ ]:
# Simulate ARIMA(1,1,1): difference once, get ARMA(1,1)
n = 300
ar = np.array([1, -0.7])
ma = np.array([1, 0.4])
arma_process = ArmaProcess(ar, ma)
stationary = arma_process.generate_sample(nsample=n)
# Integrate (cumulative sum) to make I(1)
arima_data = np.cumsum(stationary)

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# Row 1: Original I(1) series
axes[0, 0].plot(arima_data, color='#1A3A6E', linewidth=1)
axes[0, 0].set_title('ARIMA(1,1,1) Series', fontweight='bold')
axes[0, 0].set_xlabel('Time')

plot_acf(arima_data, lags=20, ax=axes[0, 1])
axes[0, 1].set_title('ACF: I(1) - Slow Decay', fontweight='bold')

plot_pacf(arima_data, lags=20, ax=axes[0, 2])
axes[0, 2].set_title('PACF: I(1)', fontweight='bold')

# Row 2: After differencing
diff_data = np.diff(arima_data)
axes[1, 0].plot(diff_data, color='#2E7D32', linewidth=0.8)
axes[1, 0].axhline(0, color='red', linestyle='--', linewidth=0.5)
axes[1, 0].set_title('After Differencing (d=1)', fontweight='bold')
axes[1, 0].set_xlabel('Time')

plot_acf(diff_data, lags=20, ax=axes[1, 1])
axes[1, 1].set_title('ACF: ARMA(1,1) Pattern', fontweight='bold')

plot_pacf(diff_data, lags=20, ax=axes[1, 2])
axes[1, 2].set_title('PACF: ARMA(1,1) Pattern', fontweight='bold')

plt.suptitle('ARIMA(p,d,q): Difference d times, then fit ARMA(p,q)', fontweight='bold', fontsize=13)
plt.tight_layout()
save_chart(fig, 'ch3_arima_model')
plt.show()

# Fit the model
model = ARIMA(arima_data, order=(1, 1, 1)).fit()
print("Fitted ARIMA(1,1,1):")
print(f"  AR(1) coeff: {model.params[1]:.4f} (true: 0.7)")
print(f"  MA(1) coeff: {model.params[2]:.4f} (true: 0.4)")
print(f"  AIC: {model.aic:.2f}")